<a href="https://colab.research.google.com/github/TomYOON/Stock-project/blob/master/Basic_Model_v3_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import os

# **Mount your drive**
본인의 구글 드라이브와 Colab을 연동합니다.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#importing data from url

url="https://raw.githubusercontent.com/Concarne2/stock_data/master/DATA/sk%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4(%EC%88%98%EC%A0%95).csv"
csv_df=pd.read_csv(url)

#  0     1    2    3     4      5    6    7
# year,month,day,close,volume,start,high,low


csv_np_raw = csv_df.to_numpy()
row,col = np.shape(csv_np_raw)

for i in range(row):
  for j in range(3,col):
    csv_np_raw[i][j] = csv_np_raw[i][j].replace(",","")

full_data_np = np.flip(csv_np_raw.astype(int),axis=0)
data_np = full_data_np[:,[5,3,6,7,4]]
#data_np = full_data_np[:,[6,3,7,4]]

print(data_np.shape)
#plt.plot(data_np[:,0])

(980, 5)


In [0]:
#normalize data

#data_min = np.min(data_np[:,1])
#data_scale = np.max(data_np[:,1]) - np.min(data_np[:,1])

scaler = MinMaxScaler()
scaler.fit(data_np)
data_norm = scaler.transform(data_np)
data_norm = np.insert(data_norm,0,np.arange(data_norm.shape[0]),axis=1)
#data = torch.from_numpy(data_norm)
#plt.plot(data_norm[:,0])

In [0]:
class MoreBasicLSTM(nn.Module):   
  def __init__(self, input_size, hidden, drop, output_size):
    super().__init__()
    self.date = output_size
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden)
    self.drop = nn.Dropout(p = drop)
    self.fclayers = nn.Sequential(
        nn.Softsign(),
        nn.Linear(in_features = hidden, out_features = output_size)
    )
    
    
  def forward(self, inputs):
    
    _,(hidden,_) = self.lstm(inputs)
    output = self.drop(hidden.squeeze(0))
    output = self.fclayers(output).squeeze(0)
    
    return output

In [0]:
class MoreBasicLSTM_timestamp_processing(nn.Module):   
  def __init__(self, input_size, hidden, drop, output_size):
    super().__init__()
    self.date = output_size
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden)
    self.drop = nn.Dropout(p = drop)
    self.fclayers = nn.Sequential(
        nn.Softsign(),
        nn.Linear(in_features = hidden, out_features = output_size)
    )
    
    
  def forward(self, inputs):
    input_without_timestamp = inputs[:,:,1:]
    batch_size = inputs
    
    _,(hidden,_) = self.lstm(input_without_timestamp)
    output = self.drop(hidden.squeeze(0))
    output = self.fclayers(output).squeeze(0)
    print(output.size())
    output = output.view(self.date,1)
    timestamps = torch.from_numpy(np.arange(self.date) + inputs[-1,:,0].item() + 1).view(self.date,1)
    
    return torch.cat([timestamps,output],axis=1)

In [0]:
class LSTM_twolayer(nn.Module):   
  def __init__(self, input_size, hidden, drop, output_size):
    super().__init__()
    self.date = output_size
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden, num_layers = 2, dropout = drop)
    self.drop = nn.Dropout(p = drop)
    self.fclayers = nn.Sequential(
        nn.Softsign(),
        nn.Linear(in_features = hidden, out_features = output_size)
    )
    
    
  def forward(self, inputs):
    
    _,(hidden,_) = self.lstm(inputs)
    output = self.drop(hidden[1,:,:].squeeze(0))
    output = self.fclayers(output).squeeze(0)
    
    return output

In [0]:
class LSTM_sequential(nn.Module):   
  def __init__(self, input_size, hidden, drop, num_layer=1):
    super().__init__()
    self.num_layers = num_layer
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden,num_layers = self.num_layers, dropout = drop)
    self.drop = nn.Dropout(p = drop)
    self.fclayers = nn.Sequential(
        nn.Softsign(),
        nn.Linear(in_features = hidden, out_features = 1)
    )
    
    
  def forward(self, inputs):
    
    _,(hidden,_) = self.lstm(inputs)
    output = self.drop(hidden[1,:,:].squeeze(0))
    output = self.fclayers(output).squeeze(0)
    
    return output

#str 변환시 함수 이름이 출력되게함
  def __str__(self):
    return str(LSTM_sequential).split(".")[1][:-2]

In [0]:
def train(time_steps, date, data_target_batches, model, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0
  #prediction_list=[]
  batch_amount = len(data_target_batches)
  batch_size = list(data_target_batches[0][0].size())[1]
  model.train()
  
  for data, target in data_target_batches:
    
    assert target[0,0,0].item() == (data[-1,0,0].item() + 1)
    
    optimizer.zero_grad()   
    
    data_without_timestamp = data[:,:,1:].clone()

    all_predictions = []
    
    for d in range(date):
      prediction_batch = model(data_without_timestamp)
      all_predictions.append(prediction_batch)
      data_last = data_without_timestamp[-1:,:,:].clone()
      for b in range(batch_size):
        data_last[0,b,1] = prediction_batch[b].item()  


      data_without_timestamp = torch.cat((data_without_timestamp[1:,:,:],data_last),axis=0)

    target_without_timestamp = torch.squeeze(target[:,:,1])
    loss = criterion(target_without_timestamp, torch.cat(all_predictions,axis=1))
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
  
  return epoch_loss / batch_amount

In [0]:
def evaluate(time_steps, date, data, target, model, criterion):
  epoch_loss = 0
  epoch_acc = 0
  predictions=[]
  
  model.eval()
  num_tries = list(target.size())[1] - (date - 1)
  
  with torch.no_grad():
    for i in range(num_tries):

      starting_date = data[i:i+time_steps,:,:][-1,0,0].item()

      input_data = data[i:i+time_steps,:,1:]

      all_predictions = []

      for d in range(date):
        prediction = model(input_data)
        all_predictions.append(prediction)
        data_last = input_data[-1:,:,:].clone()
        data_last[0,0,1] = prediction.item()

        input_data = torch.cat((input_data[1:,:,:],data_last),axis=0)

      
      total_prediction = torch.tensor(all_predictions).to(device)
      
      timestamps = torch.from_numpy(np.arange(date) + starting_date + 1).to(device).view(date,1)
      
      prediction_with_time = torch.cat([timestamps,total_prediction.view(date,1)],axis=1)
      
      predictions.append(prediction_with_time.tolist())
      loss = criterion(torch.squeeze(target)[i:i+date,1],total_prediction)

      epoch_loss += loss.item()
    
  return epoch_loss/num_tries, np.array(predictions)

In [0]:
def view_plot(prediction_list, target_list):
  fig = plt.figure(figsize=(20, 20))

  ax = []
  for i in range(7):
    ax.append(fig.add_subplot(7,1,i+1)) 

  step = 100
  for i in range(1,8):
    start = (i-1) * step

    if i == 7: end = prediction_list.shape[0]
    else: end = i * step

  # try:
    ax[i-1].plot(target_list[0:,0][start:end],target_list[0:,1][start:end])
    ax[i-1].plot(prediction_list[:,0,0][start:end],prediction_list[:,0,1][start:end])
    for j in range(start, end):
      if j%foresee_date == 0:
        ax[i-1].plot(prediction_list[:,:,0][j],prediction_list[:,:,1][j], color = 'b')

In [0]:
def view_plot_val(prediction_list, target_list):
  fig = plt.figure(figsize=(20, 5))

  ax = []
  distribution = 2
  for i in range(distribution):
    ax.append(fig.add_subplot(distribution,1,i+1)) 

  step = 100
  for i in range(1,distribution+1):
    start = (i-1) * step

    if i == distribution: end = prediction_list.shape[0]
    else: end = i * step
  # try:
    ax[i-1].plot(target_list[0:,0][start:end],target_list[0:,1][start:end])
    ax[i-1].plot(prediction_list[:,0,0][start:end],prediction_list[:,0,1][start:end])
    for j in range(start, end):
      if j%foresee_date == 0:
        ax[i-1].plot(prediction_list[:,:,0][j],prediction_list[:,:,1][j], color = 'b')

**hyper parameter**


In [0]:
lr = 0.0005
time_steps = 20
epoch = 50
batch_size = 5
dropout_rate = 0.3
num_layers = 2
input_size = data_np.shape[1] #open close high low volume
#hidden1 = 100
hidden = 25
#fc = 15

foresee_date = 5
shuffle = True

# **create model**
지정 디렉터리에 같은 학습모델이 있을 경우, 그 모델을 불러와서 학습

1.   지정 디렉터리에 같은 학습모델이 있을 경우, 그 모델을 불러와서 학습
2.   마운트 필수



In [0]:
#def call_model(model):


In [0]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_list = [LSTM_sequential]  #여기다가 만든 모델 넣어서 자동학습 가능하게

#model = BasicLSTM(input_size, hidden1, hidden2, fc)
#model = BasicLSTM(input_size, hidden, fc)
#model = MoreBasicLSTM(input_size, hidden, dropout_rate, foresee_date)
#model = LSTM_twolayer(input_size, hidden, dropout_rate, foresee_date)
# model = model_list[0](input_size, hidden, dropout_rate, num_layers)
# model = model.to(device)
# model = model.double()



path_dir = "/content/gdrive/My Drive/Colab Notebooks/stock_project/Models/"


#optimizer = optim.SGD(model.parameters(), lr = lr)
# optimizer = optim.Adam(model.parameters(), lr = lr)
# #criterion = nn.L1Loss()
# criterion = nn.MSELoss()
# criterion = criterion.to(device)


pre_train_loss = 1

# pretrained_model_list = os.listdir(path_dir)
# pretrained_model_list.sort()   #가장 loss가 낮은 모델을 부르기 위해 정렬 #저장양식 model_name + "-loss_"+str(int(loss*1000)) ex LSTM_sequential_loss_1 #모델/loss 구분 "-"(하이푼)
# print(pretrained_model_list)
# i = 0
# for trained_model in pretrained_model_list:
#   i+=1
#   #print(trained_model.split("-")[0])
#   if str(model) == trained_model.split("-")[0] :            #저장된 모델이 지금 부르는 모델이랑 같은모델일 경우 state를 불러옴
#     model.load_state_dict(torch.load(path_dir+"/"+trained_model)["model_state"])
#     pre_train_loss = torch.load(path_dir+"/"+trained_model)["loss"]
#     lr =torch.load(path_dir+"/"+trained_model)["lr"]
#     optimzier = torch.load(path_dir+"/"+trained_model)["optimizer"]
#     print("불러온 모델",trained_model)                            #저장 조건을 위해 저장 당시의 loss를 부르고, lr 역시 이전의 lr일 경우 건너뛰기 위해 저장후 불러옴
#     #print(model.state_dict())
#     break
#   if i == len(pretrained_model_list)-1:
#     print("저장된 같은 모델이 없습니다")

#######test
# train_loss = 1
# model_name = dir+str(model)+"-"+str(num_layers)+'-'+str(hidden)+"-"+'loss_'+str(int(train_loss*1000))+".pt"         #여러명이 돌릴경우 직관성을 위해 filename에 loss도 포함
# torch.save(save_dict,model_name)  

def call_model(model, num_layer, hidden_layer):

  path_dir = "/content/gdrive/My Drive/Colab Notebooks/stock_project/Models/"
  pretrained_model_list = os.listdir(path_dir)
  pretrained_model_list.sort()   #가장 loss가 낮은 모델을 부르기 위해 정렬 #저장양식 model_name + "-loss_"+str(int(loss*1000)) ex LSTM_sequential_loss_1 #모델/loss 구분 "-"(하이푼)
  i = 0
  model_state = model.state_dict()
  pre_train_loss = 1
  for trained_model in pretrained_model_list:
    i+=1
    if str(model) == trained_model.split("-")[0] and num_layer == int(trained_model.split("-")[1]) and hidden_layer == int(trained_model.split("-")[2]) :
      model_state = torch.load(path_dir+"/"+trained_model)["model_state"]
      #model.load_state_dict(torch.load(path_dir+"/"+trained_model)["model_state"])
      pre_train_loss = torch.load(path_dir+"/"+trained_model)["loss"]
      lr =torch.load(path_dir+"/"+trained_model)["lr"]
      optimzier = torch.load(path_dir+"/"+trained_model)["optimizer"]
      print("불러온 모델",trained_model, "pre_train_loss: ",pre_train_loss)   
      break
    if i == len(pretrained_model_list):
      print("저장된 같은 모델이 없습니다")
  return model_state, pre_train_loss




In [0]:
######test######
# model_state , pre_train_loss = call_model(model, num_layers, hidden)
# model.load_state_dict(model_state)

In [0]:
def data_preprocessing(data_norm,time_steps, foresee_date):
  slice_point = 800
  data_train_np = data_norm[:slice_point,:]
  data_val_np = data_norm[slice_point:,:]
  data_train_tensor = torch.from_numpy(np.expand_dims(data_train_np, axis=1)).to(device)
  data_val_tensor = torch.from_numpy(np.expand_dims(data_val_np, axis=1)).to(device)

  target_train_np = data_train_np[time_steps:,[0,2]]

  target_train_tensor = torch.from_numpy(np.expand_dims(target_train_np, axis=0)).to(device)
  target_val_np = data_val_np[time_steps:,[0,2]]
  target_val_tensor = torch.from_numpy(np.expand_dims(target_val_np, axis=0)).to(device)

  train_batches = []

  for i in range((slice_point - time_steps - (foresee_date - 1)) // batch_size):
    data_batch = []
    train_batch = []
    for b in range(batch_size):
      data_batch.append(data_train_tensor[i*batch_size + b:i*batch_size + b + time_steps,:,:])
      train_batch.append(target_train_tensor[:,i*batch_size + b:i*batch_size + b + foresee_date])
      
    train_batches.append((torch.cat(data_batch,axis=1),torch.cat(train_batch,axis=0)))

  return data_train_np, data_val_np, target_train_np, target_val_np, train_batches ,data_batch, data_val_tensor, data_train_tensor,target_train_tensor,target_val_tensor

In [0]:
# data_train_np, data_val_np, target_train_np, target_val_np, train_batches, data_batch = data_preprocessing(data_norm, time_steps, foresee_date)
# train(time_steps, foresee_date, train_batches, model, optimizer, criterion)

In [0]:
# slice_point = 800

# data_train_np = data_norm[:slice_point,:]

# data_val_np = data_norm[slice_point:,:]
# data_train_tensor = torch.from_numpy(np.expand_dims(data_train_np, axis=1)).to(device)
# data_val_tensor = torch.from_numpy(np.expand_dims(data_val_np, axis=1)).to(device)

# target_train_np = data_train_np[time_steps:,[0,2]]

# target_train_tensor = torch.from_numpy(np.expand_dims(target_train_np, axis=0)).to(device)
# target_val_np = data_val_np[time_steps:,[0,2]]
# target_val_tensor = torch.from_numpy(np.expand_dims(target_val_np, axis=0)).to(device)

# train_batches = []

# for i in range((slice_point - time_steps - (foresee_date - 1)) // batch_size):
#   data_batch = []
#   train_batch = []
#   for b in range(batch_size):
#     data_batch.append(data_train_tensor[i*batch_size + b:i*batch_size + b + time_steps,:,:])
#     train_batch.append(target_train_tensor[:,i*batch_size + b:i*batch_size + b + foresee_date])
    
#   train_batches.append((torch.cat(data_batch,axis=1),torch.cat(train_batch,axis=0)))

# = data_preprocessing(data_norm)
  

# **Model training**
이전 모델의 loss보다 낮은 loss를 기록할 경우 자동 저장

In [0]:
save_point = 1       ##mamximum = 5040
count = 0 

In [0]:
dir = "/content/gdrive/My Drive/Colab Notebooks/stock_project/Models/"
lr_list = [lr, lr/3, lr/5]
hidden_list = list(range(10,50,5))
num_layer_list = list(range(2,4))
foresee_date_list = list(range(1,6))
time_step_list = list(range(10,31))



# optimizer_list = [optim.Adam(model.parameters(), lr = lr), optim.SGD(model.parameters(), lr = lr)]

                     
#try:
for num_layer2 in num_layer_list:
  for h_layer2 in hidden_list:
    model = LSTM_sequential(input_size, hidden, dropout_rate, num_layer2)
    model = model.to(device)
    model = model.double()
    optimizer = optim.Adam(model.parameters(), lr = lr)
    criterion = nn.MSELoss()
    criterion = criterion.to(device)
    model_state, pre_train_loss =  call_model(model, num_layer2, h_layer2)
    model.load_state_dict(model_state)
    for time_step2 in time_step_list:
      for foresee_date2 in foresee_date_list:
        data_train_np, data_val_np, target_train_np, target_val_np, train_batches ,data_batch, data_val_tensor, data_train_tensor,target_train_tensor,target_val_tensor = data_preprocessing(data_norm, time_step2,foresee_date2)
        for lr2 in lr_list:
          count +=1
          #print(count)
          if count < save_point: continue         #저장지점까지 continue

          # if lr < lr2 and lr != lr_list[-1]:      #만약 불러온 모델의 lr이 더 낮을경우 그 lr에 대한 학습은 건너뜀, 제일 작은 lr경우는 학습함
          #   continue
          print("lr:",lr2, "foresee_date:",foresee_date2, "h_layer:",h_layer2, "num_layer:",num_layer2)
          if shuffle:
            random.shuffle(train_batches)
            for ep in range(100):
              optimizer = optim.Adam(model.parameters(), lr = lr2)
              start_time = time.time()
              train_loss = train(time_step2, foresee_date2, train_batches, model, optimizer, criterion)
              end_time = time.time()
              if ep%10 == 0:
                print(train_loss)
                print(end_time - start_time)
                if train_loss < pre_train_loss and train_loss < 0.003:      #이전보다 loss가 낮으면 저장함.
                  pre_train_loss = train_loss           #제일 낮은 loss 갱신
                  if not os.path.exists(dir):
                    os.makedirs(dir)          #dir가 없으면 만들어서 저장
                  save_dict = {"model_state":model.state_dict(),
                                "loss":train_loss,
                                "lr": lr2,
                                "optimizer": optimizer,
                                "hidden_layer": h_layer2,
                                "num_layers" : num_layer2,
                                "time_step":time_step2,
                                "foresee_date":foresee_date2}
                  model_name = dir+str(model)+"-"+str(num_layer2)+'-'+str(h_layer2)+"-"+'loss_'+str(int(train_loss*1000))+".pt"         #여러명이 돌릴경우 직관성을 위해 filename에 loss도 포함
                  torch.save(save_dict,model_name)
                  print("save",model_name)
                  loss_train, predictions_train = evaluate(time_step2, foresee_date2, data_train_tensor, target_train_tensor, model, criterion)
                  view_plot(predictions_train, target_train_np)
                  loss_val, predictions_val = evaluate(time_step2,foresee_date2, data_val_tensor, target_val_tensor, model, criterion)
                  view_plot_val(predictions_val, target_val_np)
# except Exception as inst:
#   print(inst)
#   print("your check point:", count)

  

불러온 모델 LSTM_sequential-2-10-loss_2.pt pre_train_loss:  0.002111304959446665
lr: 0.0005 foresee_date: 1 h_layer: 10 num_layer: 2


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0.0024017244901398814
0.6377215385437012
0.002310123177862386
0.6220149993896484
0.002484726599044893
0.6147279739379883
0.002550313248962323
0.6135392189025879
0.0026887682078099947
0.6161890029907227
0.0025237981778805435
0.6285994052886963
0.0023294144798994038
0.6241064071655273
0.002336970703972355
0.5996582508087158
0.002408729352007488
0.6075229644775391
0.0025232643379691565
0.613436222076416
lr: 0.00016666666666666666 foresee_date: 1 h_layer: 10 num_layer: 2
0.0023857637018573864
0.6216869354248047
0.00221907660242061
0.6214368343353271
0.002355576273970029
0.6037538051605225
0.002194492711615605
0.6193337440490723
0.0022660368381096007
0.6191487312316895
0.002451359743644762
0.6271615028381348
0.0022402746594114664
0.6077039241790771
0.0020989794548117407
0.5985569953918457
save /content/gdrive/My Drive/Colab Notebooks/stock_project/Models/LSTM_sequential-2-10-loss_2.pt
0.00237914694420562
0.6279683113098145
0.002289699671872271
0.6153607368469238
lr: 0.0001 foresee_date: 1 h

In [0]:
#direc = "/content/gdrive/My Drive/Colab Notebooks/stock_project/Models/model1.py"
#torch.save(model.state_dict(), direc)

In [0]:
#model2 = LSTM_sequential(input_size, hidden, dropout_rate).to(device).double()
#model2.load_state_dict(torch.load("/content/gdrive/My Drive/Colab Notebooks/stock_project/Models/model1.py"))

In [0]:
loss_train, predictions_train = evaluate(time_steps, foresee_date, data_train_tensor, target_train_tensor, model, criterion)

# plt.plot(target_train_np[:,1])
# plt.plot(predictions_train[:,0,1])
view_plot(predictions_train, target_train_np)

NameError: ignored

In [0]:
loss_train, predictions_train = evaluate(time_steps, foresee_date, data_train_tensor, target_train_tensor, model, criterion)

d = 1
#plt.figure(figsize=(100, 5))

view_plot(predictions_train, target_train_np)



In [0]:
#print(target_train_np[d - 1:,1])
#print(predictions_train[:,d-1,1])
#print(pre_with_time)
# print(predictions_train[:,:,1][:100])
# print(predictions_train[:,:,0][0])
#print(predictions_train[:2,:,1])
# print(predictions_train.shape[0])

In [0]:
loss_train, predictions_train = evaluate(time_steps,foresee_date, data_train_tensor, target_train_tensor, model, criterion)

d = 5
view_plot(predictions_train, target_train_np)
# plt.plot(target_train_np[d - 1:,1])
# plt.plot(predictions_train[:,d - 1,1])

In [0]:
loss_val, predictions_val = evaluate(time_steps,foresee_date, data_val_tensor, target_val_tensor, model, criterion)

d = 1

view_plot_val(predictions_val, target_val_np)
# plt.plot(target_val_np[d - 1:,1])
# plt.plot(predictions_val[:,d - 1,1])

In [0]:
predictions_val.shape

In [0]:
loss_val, predictions_val = evaluate(time_steps,foresee_date, data_val_tensor, target_val_tensor, model, criterion)

d = 5
view_plot_val(predictions_val, target_val_np)
# plt.plot(target_val_np[d - 1:,1])
# plt.plot(predictions_val[:,d - 1,1])

In [0]:
optimizer = optim.Adam(model.parameters(), lr = lr/3)

for ep in range(200):
  start_time = time.time()
  if shuffle:
    random.shuffle(train_batches)
  train_loss = train(time_steps,foresee_date, train_batches, model, optimizer, criterion)
  end_time = time.time()
  if ep%50 == 0:
    print(train_loss)



In [0]:
loss_train, predictions_train = evaluate(time_steps,foresee_date, data_train_tensor, target_train_tensor, model, criterion)

d = 1
view_plot(predictions_train, target_train_np)
# plt.plot(target_train_np[d - 1:,1])
# plt.plot(predictions_train[:,d - 1,1])

In [0]:
loss_train, predictions_train = evaluate(time_steps,foresee_date, data_train_tensor, target_train_tensor, model, criterion)

d = 5
view_plot(predictions_train, target_train_np)


In [0]:
loss_val, predictions_val = evaluate(time_steps,foresee_date, data_val_tensor, target_val_tensor, model, criterion)

d = 1
view_plot_val(predictions_val, target_val_np)
# plt.plot(target_val_np[d - 1:,1])
# plt.plot(predictions_val[:,d - 1,1])

In [0]:
loss_val, predictions_val = evaluate(time_steps,foresee_date, data_val_tensor, target_val_tensor, model, criterion)

d = 5
view_plot_val(predictions_val, target_val_np)
# plt.plot(target_val_np[d - 1:,1])
# plt.plot(predictions_val[:,d - 1,1])

In [0]:
optimizer = optim.Adam(model.parameters(), lr = lr/5)

for ep in range(200):
  start_time = time.time()
  train_loss = train(time_steps,foresee_date, train_batches, model, optimizer, criterion)
  end_time = time.time()
  if ep%50 == 0:
    print(train_loss)

In [0]:
loss_train, predictions_train = evaluate(time_steps,foresee_date, data_train_tensor, target_train_tensor, model, criterion)

d = 5

# print(predictions_train.shape)
# print(target_train_np.shape)

# plt.plot(target_train_np[d - 1:,1][300:500])
# plt.plot(predictions_train[:,d - 1,1][300:500])

# print(target_train_np[d - 1,0])
# print(predictions_train[0,d-1,0])
view_plot(predictions_train, target_train_np)


In [0]:
loss_val, predictions_val = evaluate(time_steps, foresee_date,data_val_tensor, target_val_tensor, model, criterion)

# d = 5

# plt.plot(target_val_np[d - 1:,1])
# plt.plot(predictions_val[:,d - 1,1])
view_plot_val(predictions_val, target_val_np)
